# Tokenization & Embeddings — Teaching Notebook

This notebook contains runnable examples used in the lecture: bytes/encodings, `ord()`, simple BPE toy implementation, sentence tokenization (for RAG chunking), Hugging Face tokenizer demos, and a short embeddings / retrieval demo. **Run cells one-by-one** in a Python environment with internet to install required packages.

**Note:** The notebook includes `pip install` cells for convenience.

In [ ]:
# Install required packages (run this cell first)
# You may already have some of these installed. If running in a restricted environment, skip installs and run applicable cells.
!pip install --quiet transformers sentence-transformers nltk scikit-learn tiktoken
# For nltk sentence tokenizer
import nltk

nltk.download('punkt')
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

## 1) Bytes, `ord()` and encodings

Show how Python represents strings as code points and bytes.

In [ ]:
text = "I love pizza 🍕"
print("Raw string:", text)
print("\nUsing ord() on each character:")
for ch in text:
    print(repr(ch), "->", ord(ch))
print("\nUTF-8 byte sequence:")
print(list(text.encode('utf-8')))
print("\nUTF-32 byte sequence (fixed-width per code point):")
print(list(text.encode('utf-32'))[:40])  # show prefix

Raw string: I love pizza 🍕

Using ord() on each character:
'I' -> 73
' ' -> 32
'l' -> 108
'o' -> 111
'v' -> 118
'e' -> 101
' ' -> 32
'p' -> 112
'i' -> 105
'z' -> 122
'z' -> 122
'a' -> 97
' ' -> 32
'🍕' -> 127829

UTF-8 byte sequence:
[73, 32, 108, 111, 118, 101, 32, 112, 105, 122, 122, 97, 32, 240, 159, 141, 149]

UTF-32 byte sequence (fixed-width per code point):
[255, 254, 0, 0, 73, 0, 0, 0, 32, 0, 0, 0, 108, 0, 0, 0, 111, 0, 0, 0, 118, 0, 0, 0, 101, 0, 0, 0, 32, 0, 0, 0, 112, 0, 0, 0, 105, 0, 0, 0]


## 2) Sentence tokenization (useful for RAG chunking)

Use NLTK's `sent_tokenize` to split documents into sentences (or small chunks).

In [ ]:
from nltk.tokenize import sent_tokenize
doc = """It's not clocking to you. It's not clocking to you that I'm standing on business, is it.   ."""
sentences = sent_tokenize(doc)
print("Sentences:")
for i,s in enumerate(sentences, 1):
    print(i, s)

Sentences:
1 It's not clocking to you.
2 It's not clocking to you that I'm standing on business, is it.
3 .


## 3) Hugging Face tokenizer demos (GPT/GPT-2 byte-level BPE and BERT WordPiece examples)

`AutoTokenizer` loads the model-specific tokenizer; this is how you inspect tokens and token ids.

In [ ]:
from transformers import AutoTokenizer

# 🌟 Sample paragraph (covers words, punctuation, emojis, and code)
text = """ChatGPT is unbelievably powerful!
It can write code like:
def greet(name):
    print(f"Hello, {name}! 👋")"""

# ---------- GPT-2 (Byte-level BPE) ----------
print("=== GPT-2 / Byte-level BPE (tiktoken-like) ===")
tok_gpt2 = AutoTokenizer.from_pretrained("gpt2")

toks = tok_gpt2.tokenize(text)
ids = tok_gpt2.convert_tokens_to_ids(toks)

print("\nOriginal Text:\n", text)
print("\nTokens:\n", toks)
print("\nToken IDs:\n", ids)
print("\nNumber of tokens:", len(toks))

# ---------- BERT (WordPiece) ----------


# print("\n=== BERT / WordPiece ===")
# tok_bert = AutoTokenizer.from_pretrained("bert-base-uncased")

# toks_bert = tok_bert.tokenize(text)
# ids_bert = tok_bert.convert_tokens_to_ids(toks_bert)

# print("\nTokens:\n", toks_bert)
# print("\nToken IDs:\n", ids_bert)
# print("\nNumber of tokens:", len(toks_bert))



=== GPT-2 / Byte-level BPE (tiktoken-like) ===


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]


Original Text:
 ChatGPT is unbelievably powerful! 
It can write code like:
def greet(name): 
    print(f"Hello, {name}! 👋")

Tokens:
 ['Chat', 'G', 'PT', 'Ġis', 'Ġunbelievably', 'Ġpowerful', '!', 'Ġ', 'Ċ', 'It', 'Ġcan', 'Ġwrite', 'Ġcode', 'Ġlike', ':', 'Ċ', 'def', 'Ġgreet', '(', 'name', '):', 'Ġ', 'Ċ', 'Ġ', 'Ġ', 'Ġ', 'Ġprint', '(', 'f', '"', 'Hello', ',', 'Ġ{', 'name', '}', '!', 'ĠðŁĳ', 'ĭ', '")']

Token IDs:
 [30820, 38, 11571, 318, 48943, 3665, 0, 220, 198, 1026, 460, 3551, 2438, 588, 25, 198, 4299, 12589, 7, 3672, 2599, 220, 198, 220, 220, 220, 3601, 7, 69, 1, 15496, 11, 1391, 3672, 92, 0, 50169, 233, 4943]

Number of tokens: 39


**Note:** This toy BPE shows the *idea* of finding frequent adjacent pairs and merging. Real BPE implementations are optimized and trained on much larger corpora; they also handle whitespace, bytes, and special tokens.

## 5) Embeddings demo (sentence-transformers) and simple semantic retrieval

Compute sentence embeddings and perform cosine-similarity based retrieval (the retriever part of RAG).

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
from numpy.linalg import norm

model = SentenceTransformer('all-MiniLM-L6-v2')  # small and fast
docs = [
    "Mars is known as the Red Planet.",
    "Saturn has beautiful rings made of ice and rock.",
    "The Earth orbits the Sun.",
    "Mars has been a focus of robotic missions."
]
embs = model.encode(docs)
print(embs)

# simple retrieval
def most_similar(query, docs, embs, top_k=2):
    q_emb = model.encode([query])[0]
    sims = [np.dot(q_emb, e) / (norm(q_emb)*norm(e)) for e in embs]
    pairs = sorted(enumerate(sims), key=lambda x: x[1], reverse=True)[:top_k]
    return [(docs[i], score) for i,score in pairs]

print(most_similar("Which planet is red?", docs, embs))

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[[ 0.07176256 -0.01638938 -0.07682189 ...  0.01278931 -0.01877984
   0.06320196]
 [-0.02329944 -0.02525921  0.0643563  ... -0.02290123 -0.0290333
   0.05991274]
 [ 0.04201508  0.06371279  0.05456807 ...  0.02907257  0.0134282
  -0.00786757]
 [ 0.08269827 -0.04288797 -0.01748822 ...  0.02932361 -0.04017614
  -0.02907068]]
[('Mars is known as the Red Planet.', np.float32(0.7611022)), ('The Earth orbits the Sun.', np.float32(0.4303342))]
